In [287]:
%run 02c_Preprocessing_and_feature_engineering.ipynb

In [288]:
serving = pd.read_csv('../datasets/test.csv')


## Fit the whole model

In [289]:
X_3_si = si.fit_transform(X_3_use)

In [291]:
Z_3 = sc.fit_transform(X_3_si)

In [292]:
ridge_cv.fit(Z_3, y3)

RidgeCV(alphas=array([1.00000000e-02, 1.12332403e-02, 1.26185688e-02, 1.41747416e-02,
       1.59228279e-02, 1.78864953e-02, 2.00923300e-02, 2.25701972e-02,
       2.53536449e-02, 2.84803587e-02, 3.19926714e-02, 3.59381366e-02,
       4.03701726e-02, 4.53487851e-02, 5.09413801e-02, 5.72236766e-02,
       6.42807312e-02, 7.22080902e-02, 8.11130831e-02, 9.11162756e-02,
       1.02353102e-01, 1.14975700e-0...
       6.89261210e+01, 7.74263683e+01, 8.69749003e+01, 9.77009957e+01,
       1.09749877e+02, 1.23284674e+02, 1.38488637e+02, 1.55567614e+02,
       1.74752840e+02, 1.96304065e+02, 2.20513074e+02, 2.47707636e+02,
       2.78255940e+02, 3.12571585e+02, 3.51119173e+02, 3.94420606e+02,
       4.43062146e+02, 4.97702356e+02, 5.59081018e+02, 6.28029144e+02,
       7.05480231e+02, 7.92482898e+02, 8.90215085e+02, 1.00000000e+03]),
        cv=10)

## Create X Serving

In [293]:
replace_NaN_with_NA(serving, ['Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Fence'])

In [294]:
binarize_to_most_common_str_val(serving, [('Street', 'Paved Street'),
 ('Utilities', 'Has All Utilities'),
 ('Land Slope', 'Gentle Slope'),
 ('Central Air', None),
 ('Paved Drive', None)])

In [295]:
convert_num_to_cat(serving, ['PID', 'MS SubClass', 'Overall Qual', 'Overall Cond', 'Mo Sold'])

In [296]:
convert_years_to_ages(serving, 2010, [
    ('Year Built', 'Years since built'),
    ('Year Remod/Add', 'Years since remodel'),
    ('Yr Sold', 'Years since last sold')
])

In [273]:
# features_to_use_list.remove('Overall Qual_10')


In [297]:
serving.drop('PID', axis = 1 , inplace = True)

In [298]:
serving_dummy = pd.get_dummies(serving, columns = non_numerics.columns, drop_first = True)

Why is the line of code below here? This feature was present in both test and train but there were none in the "unseen" data! This caused a mismatch in columns and so I needed to add it in order for everything to line up

In [276]:
# serving_dummy['Overall Qual_10'] = 0

In [299]:
X_serving = serving_dummy.loc[: , list(X_3_use.columns)]

## Transform X Serving

In [300]:
X_serving_si = si.transform(X_serving)

In [301]:
Z_serving = sc.transform(X_serving_si)

## Generate Predictions

In [302]:
salesprices = ridge_cv.predict(Z_serving)

In [303]:
pd.DataFrame(salesprices).describe()

,0
count,878.000000
mean,179720.197886
std,68805.433555
min,57639.239141
25%,129802.570974
50%,163864.336280
75%,217035.742138
max,476182.167558


In [304]:
salesprices.shape

(878,)

In [305]:
serving['SalePrice'] = salesprices

In [306]:
submission3 = serving.loc[:, ['Id','SalePrice']]

In [307]:
submission3.set_index('Id', inplace = True)

In [310]:
submission3.to_csv('../datasets/submission3_ols.csv')

In [309]:
submission3.head()

,SalePrice
Id,
2658,122899.772308
2718,160381.834347
2414,224027.815516
1989,121611.415179
625,187867.683601
